<a href="https://colab.research.google.com/github/falconlee236/DeepLearningFrom_Scratch/blob/main/Book_2/ch08/Chapter_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8 Attention